In [ ]:
import pandas as pd
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/mount')

Mounted at /content/drive


In [ ]:
path_to_raw_data = "/content/drive/MyDrive/datasets/avito/raw_data"
category_data = pd.read_csv(path_to_raw_data+"/category.csv",delimiter = ',')
train_data = pd.read_csv("datasets/avito/raw_data/train.csv",delimiter = ',')
train_data.head(20)

In [ ]:
#предобработка текста
!pip install ufal.udpipe
#NOTE: обработка занимает довольно много времени, если делать ее с компьютера
#!cat  "datasets/avito/raw_data/categories.csv" | python3 preprocessing.py > "datasets/avito/preprocessed_data/prepr_categories.csv"
#!cat "datasets/avito/raw_data/queries.csv" | python3 preprocessing.py > "datasets/avito/preprocessed_data/prepr_queries.csv"
#!cat "datasets/avito/raw_data/description.csv" | python3 preprocessing.py > "datasets/avito/preprocessed_data/prepr_descriptions.csv"

In [ ]:
path_to_prepr_data = "/content/drive/MyDrive/sharing/datasets/avito/preprocessed_data"
prepr_queries_data = pd.read_csv(path_to_prepr_data + "/prepr_queries.csv", header = None)

In [ ]:
prepr_descriptions_data = pd.read_csv(path_to_prepr_data + "/prepr_descriptions.csv", header = None,\
                                      sep ='<<', engine = 'python')

In [ ]:
!pip uninstall gensim
!pip install gensim
!pip install --upgrade gensim

Uninstalling gensim-3.8.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gensim-3.8.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gensim/*
Proceed (y/n)? y
  Successfully uninstalled gensim-3.8.3
  Using cached https://files.pythonhosted.org/packages/5c/4e/afe2315e08a38967f8a3036bbe7e38b428e9b7a90e823a83d0d49df1adf5/gensim-3.8.3-cp37-cp37m-manylinux1_x86_64.whl


Requirement already up-to-date: gensim in /usr/local/lib/python3.7/dist-packages (3.8.3)


In [1]:
from multiprocessing import cpu_count
import gensim
from gensim import corpora
from gensim.utils import simple_preprocess
from smart_open import smart_open
import os
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
from pprint import pprint
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
#from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
from gensim.matutils import softcossim

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


ImportError: ignored

In [ ]:
# Tokenization and creation of a dictionary
class DocsToDictionary(object):
    def __init__(self, path_to_dir):
        self.path_to_dir = path_to_dir
    def __iter__(self):
        for filename in os.listdir(self.path_to_dir):
            for line in open(os.path.join(self.path_to_dir, filename), encoding='utf-8'):
                text = line.split(' ')
                text = [word for word in text if word not in stop_words]
                pline = " ".join(text)
                yield simple_preprocess(pline)
                
path_to_dir = path_to_prepr_data
main_dictionary = corpora.Dictionary(DocsToDictionary(path_to_dir))
print(main_dictionary.token2id['шнур'])


2697


In [ ]:
main_corpus = {}
class CreateCorpus(object):
    def __init__(self, path_to_dir, dictionary):
        self.path_to_dir = path_to_dir
        self.dictionary = dictionary
    def __iter__(self):
        for filename in os.listdir(self.path_to_dir):
            #global main_dictionary  # OPTIONAL, only if updating the source dictionary.
            for line in smart_open(os.path.join(self.path_to_dir, filename), encoding='utf-8'):
                # tokenize
                tokenized_list = simple_preprocess(line, deacc=True, min_len=3)
                # create bag of words
                bow = self.dictionary.doc2bow(tokenized_list, allow_update=True)
                # update the source dictionary (OPTIONAL)
                #main_dictionary.merge_with(self.dictionary)
                # lazy return the BoW
                yield bow
            
# Create the Corpus
main_corpus = CreateCorpus(path_to_dir, dictionary=main_dictionary)

In [ ]:
#main_dictionary.save('/content/drive/MyDrive/sharing/dicts/main_dictionary.dict')  # save dict to disk
main_dictionary = corpora.Dictionary.load('/content/drive/MyDrive/sharing/dicts/main_dictionary.dict')
#corpora.MmCorpus.serialize('/content/drive/MyDrive/sharing/dicts/main_corpus.mm', main_corpus)  # save corpus to disk
corpus = corpora.MmCorpus('/content/drive/MyDrive/sharing/dicts/main_corpus.mm') #loading

In [ ]:
prepr_queries = prepr_queries_data.values.flatten().tolist()
prepr_descriptions = prepr_descriptions_data.values.flatten().tolist()
tokenized_list = []
for line in prepr_descriptions:
  text = line.split(' ')
  text = [word for word in text if word not in stop_words]
  tokenized_list.append(text)
#tfidf = TfidfModel(dictionary=main_dictionary)
model = gensim.models.Word2Vec(tokenized_list, size = 300, window = 50, workers=cpu_count(), seed=12345)

In [ ]:
model.wv.most_similar('стол')

[('столешница', 0.6626424789428711),
 ('обеденный', 0.6027965545654297),
 ('столовый', 0.5398345589637756),
 ('столик', 0.5388284921646118),
 ('стол-трансформер', 0.529141902923584),
 ('табурет', 0.4932760000228882),
 ('стул', 0.4874957203865051),
 ('журнальный', 0.4848085045814514),
 ('парта', 0.47468066215515137),
 ('раздвижная', 0.4712746739387512)]

In [ ]:
model.save('/content/drive/MyDrive/sharing/models/word2vec.model')

In [ ]:
from gensim.models import Word2Vec
import scipy
from gensim.similarities import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix
#similarity_index = WordEmbeddingSimilarityIndex(w2v_model.wv)
#similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=100)

ImportError: ignored

In [ ]:

model.wv.most_similar("продавать")
def softcossim(query, documents):
    # Compute Soft Cosine Measure between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = SoftCosineSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        similarity_matrix)
    similarities = index[query]
    return similarities


NameError: ignored

In [ ]:

# lib_path='/content/libraries'
# os.symlink('/content/drive/MyDrive/Colab Notebooks', lib_path)
# sys.path.insert(0, lib_path) 
#!pip install wget 
import sys, os, zipfile, wget
#model_url = 'http://panchenko.me/data/dsl-backup/w2v-ru/all.norm-sz500-w10-cb0-it3-min5.w2v'
#m = wget.download(model_url)

w2v_fpath = "all.norm-sz500-w10-cb0-it3-min5.w2v"
w2v = gensim.models.KeyedVectors.load_word2vec_format(w2v_fpath, binary=True, unicode_errors='ignore')
for word, score in w2v.most_similar(u"дерево"):
    print (word, score)

In [ ]:
import zipfile
model_url = 'http://vectors.nlpl.eu/repository/11/213.zip'
m = wget.download(model_url)
model_file = model_url.split('/')[-1]
with zipfile.ZipFile(model_file, 'r') as archive:
    stream = archive.open('model.bin')
    model = gensim.models.KeyedVectors.load_word2vec_format(stream, binary=True)

In [ ]:
import gensim.downloader as api
dataset = api.load("text8")
data = [d for d in dataset]

[==================================================] 100.0% 31.6/31.6MB downloaded


In [ ]:
print("query=", prepr_queries[2])
print("doc=", tokenized_list[2])



query= домашний миня баня
doc= ['мини', 'баня', 'минять', 'сауна', 'предназначать', 'принятие', 'тепловой', 'процедура', 'бытовой', 'условие', 'дома', 'дача', 'также', 'спортивный', 'оздоровительный', 'косметический', 'лечебный', 'организация', 'учреждение', 'оказывать', 'общеукрепляющий', 'профилактический', 'действие', 'организм', 'человек', 'номинальный', 'напряжение', 'переменный', 'ток', 'xxx', 'максимальный', 'мощность', 'xxxx', 'вт', 'максимальный', 'температура', 'нагреть', 'воздух', 'термочехл', 'градус', 'xx', 'xxx', 'время', 'достижение', 'рабочий', 'температура', '1,5', 'минута', 'время', 'достижение', 'максимальный', 'температура', 'xx', 'минута', 'габаритный', 'размер', 'тепловой', 'камера', 'рабочий', 'состояние', 'мм', 'длина', '950', 'ширина', '900', 'высота', '1100', 'масса', '6,3', '7,3', 'килограмм', 'мини', 'баня', 'становиться', 'отличный', 'подарок', 'мужчина', 'женщина']
